In [2]:
import os
from langchain_openai import ChatOpenAI
import langsmith
from langsmith.evaluation import EvaluationResult, run_evaluator
from langchain.smith import RunEvalConfig
from dotenv import load_dotenv
load_dotenv('/Users/clarencechan/Documents/agentic-rag/process/.env')

/Users/clarencechan/Documents/agentic-rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

#### URL - https://levelup.gitconnected.com/implementing-12-ai-agent-evaluation-techniques-using-langsmith-507d5bf5c0aa

In [9]:
model = 'gpt-5-mini'
client = langsmith.Client()

#### Exact matching based evaluation
- Typiecall used for RAG or AI Agent tasks when the exepcted output is deterministic
- Fact-based QA
- Close-ended question
- Tool usage outputs
- Structured outputs

In [13]:
dataset_name = 'test-dataset'
ds = client.read_dataset(dataset_name=dataset_name)

# ds = client.create_dataset(
#     dataset_name=dataset_name,
#     description='Test dataset for tutorial & learning purposes.',
# )

# client.create_examples(
#     inputs=[
#         {
#             "prompt_template": "State the year of the declaration of independence. Respond with just the year in digits, nothing else"
#         },
#         {
#             "prompt_template": "What's the average speed of an unladen swallow?"
#         },
#     ],
#     # List of corresponding outputs.
#     outputs=[
#         {"output": "1776"},
#         {"output": "5"}
#     ],
#     dataset_id=ds.id,
# )

{'example_ids': ['17e3104e-cad3-43ed-841a-a74f29eef1a7',
  'b8394435-c9f2-4504-9aea-58af001648c3'],
 'count': 2}

In [14]:
# This is our "system under test". It takes an input dictionary,
# invokes the specified ChatOpenAI model, and returns the output in a dictionary.
def predict_result(input_: dict) -> dict:
    # The input dictionary for this function will have the key "prompt_template"
    # which matches the key we defined in our dataset's inputs.
    prompt = input_["prompt_template"]
    
    # Initialize and call the model
    response = ChatOpenAI(model=model, temperature=0).invoke(prompt)
    
    # The output key "output" matches the key in our dataset's outputs for comparison.
    return {"output": response.content}

# Build Evaluators
# Built-in exact_match evaluator
# Custom compare_label evaluator using @run_evaluator
# The @run_evaluator decorator registers this function as a custom evaluator
@run_evaluator
def compare_label(run, example) -> EvaluationResult:
    """
    A custom evaluator that checks for an exact match.
    
    Args:
        run: The LangSmith run object, which contains the model's outputs.
        example: The LangSmith example object, which contains the reference data.
    
    Returns:
        An EvaluationResult object with a key and a score.
    """
    # Get the model's prediction from the run's outputs dictionary.
    # The key 'output' must match what our `predict_result` function returns.
    prediction = run.outputs.get("output") or ""
    
    # Get the reference answer from the example's outputs dictionary.
    # The key 'output' must match what we defined in our dataset.
    target = example.outputs.get("output") or ""
    
    # Perform the comparison.
    match = prediction == target
    
    # Return the result. The key is how the score will be named in the results.
    # The score for exact match is typically binary (1 for a match, 0 for a mismatch).
    return EvaluationResult(key="matches_label", score=int(match))

In [15]:
# This defines the configuration for our evaluation run.
# Specoify both custom and built in evaluaators - every model run will be scored by both
eval_config = RunEvalConfig(
    # We can specify built-in evaluators by their string names.
    evaluators=["exact_match"], 
    
    # We pass our custom evaluator function directly in a list.
    custom_evaluators=[compare_label],
)

# This command triggers the evaluation.
# It will run the `predict_result` function for each example in the dataset
# and then score the results using the evaluators in `eval_config`.
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=predict_result,
    evaluation=eval_config,
    verbose=True, # This will print the progress bar and links
    project_metadata={"version": "1.0.1", "model": model}, # Optional metadata for the project
)

View the evaluation results for project 'damp-position-49' at:
https://smith.langchain.com/o/447a3e6d-e186-494e-9c8e-c4650326fd0d/datasets/98e8b24c-e8f7-4fc4-b539-64e59cc3348e/compare?selectedSessions=1300f7ea-cd5d-4f65-8939-8a8d399b9072

View all tests for Dataset test-dataset at:
https://smith.langchain.com/o/447a3e6d-e186-494e-9c8e-c4650326fd0d/datasets/98e8b24c-e8f7-4fc4-b539-64e59cc3348e
[------------------------------------------------->] 2/2


,feedback.exact_match,feedback.matches_label,error,execution_time,run_id
count,2.000000,2.000000,0,2.000000,2
unique,NaN,NaN,0,NaN,2
top,NaN,NaN,NaN,NaN,42719130-216b-4df3-b198-3617881b8a32
freq,NaN,NaN,NaN,NaN,1
mean,0.500000,0.500000,NaN,6.866100,NaN
std,0.707107,0.707107,NaN,2.975639,NaN
min,0.000000,0.000000,NaN,4.762006,NaN
25%,0.250000,0.250000,NaN,5.814053,NaN
50%,0.500000,0.500000,NaN,6.866100,NaN
75%,0.750000,0.750000,NaN,7.918148,NaN


{'project_name': 'damp-position-49',
 'results': {'17e3104e-cad3-43ed-841a-a74f29eef1a7': {'input': {'prompt_template': 'State the year of the declaration of independence. Respond with just the year in digits, nothing else'},
   'feedback': [EvaluationResult(key='exact_match', score=1, value=None, comment=None, correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('904f4139-2d76-44e1-a455-7f8e85e085ca'))}, feedback_config=None, source_run_id=None, target_run_id=None, extra=None),
    EvaluationResult(key='matches_label', score=1, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('70ab3418-d3e2-45af-be57-64cbc6136fbc'), target_run_id=None, extra=None)],
   'execution_time': 4.762006,
   'run_id': '42719130-216b-4df3-b198-3617881b8a32',
   'output': {'output': '1776'},
   'reference': {'output': '1776'}},
  'b8394435-c9f2-4504-9aea-58af001648c3': {'input': {'prompt_template': "What's the average speed of an unladen swallow?"},


#### Unstructured Q&A Eval

In [1]:
# Main difference is that our GT answers are now reference points of correctness not templates for an exact match

In [23]:
# Create the dataset in LangSmith
client = langsmith.Client()
try: 
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Q&A dataset about LangSmith documentation."
    )
except Exception as e:
    dataset = client.read_dataset(dataset_name=dataset_name)
print(f"Dataset ID = {dataset.id}")

# These are our question-and-answer examples. The answers serve as 'ground truth'.
qa_examples = [
    (
        "What is LangChain?",
        "LangChain is an open-source framework for building applications using large language models. It is also the name of the company building LangSmith.",
    ),
    (
        "How might I query for all runs in a project?",
        "You can use client.list_runs(project_name='my-project-name') in Python, or client.ListRuns({projectName: 'my-project-name'}) in TypeScript.",
    ),
    (
        "What's a langsmith dataset?",
        "A LangSmith dataset is a collection of examples. Each example contains inputs and optional expected outputs or references for that data point.",
    ),
    (
        "How do I move my project between organizations?",
        "LangSmith doesn't directly support moving projects between organizations.",
    ),
]

# Add the examples to our dataset
# The input key is 'question' and the output key is 'answer'.
# These keys must match what our RAG chain expects and produces.
for question, answer in qa_examples:
    client.create_example(
        inputs={"question": question},
        outputs={"answer": answer},
        dataset_id=dataset.id,
    )

Dataset ID = 98e8b24c-e8f7-4fc4-b539-64e59cc3348e


In [ ]:
def predict_result(input_: dict) -> dict:
    # The input dictionary for this function will have the key "prompt_template"
    # which matches the key we defined in our dataset's inputs.
    prompt = input_.get("question") or input_.get('prompt_template')
    
    # Initialize and call the model
    response = ChatOpenAI(model=model, temperature=0).invoke(prompt)
    
    # The output key "output" matches the key in our dataset's outputs for comparison.
    return {"output": response.content}

In [28]:
# Configure the evaluation to use the "qa" evaluator, which grades for
# "correctness" based on the reference answer.
eval_config = RunEvalConfig(
    evaluators=["qa"],
)

# Run the RAG chain over the dataset and apply the evaluator
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=predict_result,
    evaluation=eval_config,
    verbose=True,
    project_metadata={"version": "1.0.0", "model": "gpt-5-mini"},
)

View the evaluation results for project 'extraneous-shoe-50' at:
https://smith.langchain.com/o/447a3e6d-e186-494e-9c8e-c4650326fd0d/datasets/98e8b24c-e8f7-4fc4-b539-64e59cc3348e/compare?selectedSessions=4fa2aecc-4a93-4c7e-b621-770130ba0ef7

View all tests for Dataset test-dataset at:
https://smith.langchain.com/o/447a3e6d-e186-494e-9c8e-c4650326fd0d/datasets/98e8b24c-e8f7-4fc4-b539-64e59cc3348e


Chain failed for example 17e3104e-cad3-43ed-841a-a74f29eef1a7 with inputs {'prompt_template': 'State the year of the declaration of independence. Respond with just the year in digits, nothing else'}
Error Type: KeyError, Message: 'question'
Chain failed for example b8394435-c9f2-4504-9aea-58af001648c3 with inputs {'prompt_template': "What's the average speed of an unladen swallow?"}
Error Type: KeyError, Message: 'question'


[------------------------------------------------->] 6/6


,feedback.correctness,error,execution_time,run_id
count,4.00000,2,6.000000,6
unique,NaN,2,NaN,6
top,NaN,'question',NaN,d02e80c5-f9de-4a8c-b1a0-37a5073405dd
freq,NaN,1,NaN,1
mean,0.50000,NaN,18.364198,NaN
std,0.57735,NaN,15.018547,NaN
min,0.00000,NaN,0.000826,NaN
25%,0.00000,NaN,5.260110,NaN
50%,0.50000,NaN,22.210899,NaN
75%,1.00000,NaN,30.446117,NaN


{'project_name': 'extraneous-shoe-50',
 'results': {'f1e31e1f-a0a5-400b-9572-1647b90bcb0b': {'input': {'question': 'How do I move my project between organizations?'},
   'feedback': [EvaluationResult(key='correctness', score=0, value='INCORRECT', comment='INCORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('07f82966-2b2e-4af3-aa54-297bbc050219'))}, feedback_config=None, source_run_id=None, target_run_id=None, extra=None)],
   'execution_time': 32.961527,
   'run_id': 'd02e80c5-f9de-4a8c-b1a0-37a5073405dd',
   'output': {'output': 'Which service/platform are you using (GitHub, GitLab, Google Cloud, Azure DevOps, Bitbucket, Jira, etc.)? Exact steps depend on that — I can give a step‑by‑step for the product you use.\n\nMeanwhile, here’s a short general checklist plus quick instructions for several common platforms.\n\nGeneral checklist before you move anything\n- Confirm you have the required permissions in both source and target organizations.\n- Back up the project

#### Structured Data Comparison

In [3]:
client = langsmith.Client()

In [30]:
# The URL of the public dataset on LangSmith
dataset_url = "https://smith.langchain.com/public/08ab7912-006e-4c00-a973-0f833e74907b/d"
dataset_name = "Contract Extraction Eval Dataset"

# Clone the public dataset to your own account
client.clone_public_dataset(dataset_url, dataset_name=dataset_name)

Dataset(name='Contract Extraction Eval Dataset', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('f1f777fb-0b39-4a08-93d8-c098adc18756'), created_at=datetime.datetime(2025, 10, 3, 14, 54, 37, 411802, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 3, 14, 54, 37, 411802, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata=None)

In [4]:
from typing import List, Optional
from pydantic import BaseModel

# Define the schema for a party's address
class Address(BaseModel):
    street: str
    city: str
    state: str

# Define the schema for a party in the contract
class Party(BaseModel):
    name: str
    address: Address

# The top-level schema for the entire contract
class Contract(BaseModel):
    document_title: str
    effective_date: str
    parties: List[Party]


In [5]:
from langchain.chains import create_extraction_chain
from langchain_anthropic import ChatAnthropic
from langsmith.evaluation import LangChainStringEvaluator

# For this task, we'll use a powerful model capable of following complex instructions.
# Note: You can swap this with an equivalent OpenAI model.
llm = ChatAnthropic(model="claude-2.1", temperature=0, max_tokens=4000)

# Create the extraction chain, providing the schema and the LLM.
extraction_chain = create_extraction_chain(Contract.schema(), llm)

/var/folders/40/2qwvhqqd5x38g72sq_ktmlzm0000gn/T/ipykernel_27519/258324972.py:10: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  extraction_chain = create_extraction_chain(Contract.schema(), llm)
/var/folders/40/2qwvhqqd5x38g72sq_ktmlzm0000gn/T/ipykernel_27519/258324972.py:10: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. If you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langcha

In [8]:
# The evaluation configuration specifies our JSON-aware evaluator.
# The 'json_edit_distance' evaluator compares the structure and content of two JSON objects.
eval_config = RunEvalConfig(
    evaluators=["json_edit_distance"]
)

# # Run the evaluation
# client.run_on_dataset(
#     dataset_name='Contract Extraction Eval Dataset',
#     llm_or_chain_factory=extraction_chain,
#     evaluation=eval_config,
#     # The input key in our dataset is 'context', which we map to the chain's 'input' key.
#     input_mapper=lambda x: {"input": x["context"]},
#     # The output from the chain is a dict {'text': [...]}, we care about the 'text' value.
#     output_mapper=lambda x: x['text'],
#     verbose=True,
#     project_metadata={"version": "1.0.0", "model": "claude-2.1"},
# )

#### Dynamic Ground Truth
- E.g. based on a live database, an inventory system, or a constantly updating API, how can you create a reliable test set?
- use indirection, instead of storing the static answer, store a reference or a query that can be executed at the time of evaluation

In [10]:
# Our list of questions and the corresponding pandas code to find the answer.
questions_with_references = [
    ("How many passengers were on the Titanic?", "len(df)"),
    ("How many passengers survived?", "df['Survived'].sum()"),
    ("What was the average age of the passengers?", "df['Age'].mean()"),
    ("How many male and female passengers were there?", "df['Sex'].value_counts()"),
    ("What was the average fare paid for the tickets?", "df['Fare'].mean()"),
]

# Create a unique dataset name
dataset_name = "Dynamic Titanic QA"

client = langsmith.Client()
# Create the dataset in LangSmith
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA over the Titanic dataset with dynamic references.",
)

# Populate the dataset. The input is the question, and the output is the code.
client.create_examples(
    inputs=[{"question": q} for q, r in questions_with_references],
    outputs=[{"reference_code": r} for q, r in questions_with_references],
    dataset_id=dataset.id,
)

{'example_ids': ['8bf9e5c9-bf6c-4b91-bf0c-8498073d91c9',
  '81327d10-4780-4ebf-a87c-0a8177f7292c',
  '51abab2f-a113-4e66-b768-0b7c5c5c53da',
  'a36f07a4-9b8c-43ff-9fc7-1caae8378fb9',
  'ae57634f-a538-4fdd-9535-46492887a870'],
 'count': 5}

In [11]:
import pandas as pd

# Load the Titanic dataset from a URL
titanic_url = "https://raw.githubusercontent.com/jorisvandenbossche/pandas-tutorial/master/data/titanic.csv"
df = pd.read_csv(titanic_url)

In [12]:
# Define the LLM for the agent
llm = ChatOpenAI(model="gpt-4", temperature=0)

# This function creates and invokes the agent on the current state of `df`
def predict_pandas_agent(inputs: dict):
    # The agent is created with the current `df`
    agent = create_pandas_dataframe_agent(agent_type="openai-tools", llm=llm, df=df)
    return agent.invoke({"input": inputs["question"]})

In [13]:
from typing import Optional
from langchain.evaluation.criteria.eval_chain import LabeledCriteriaEvalChain

class DynamicReferenceEvaluator(LabeledCriteriaEvalChain):
    def _get_eval_input(
        self,
        prediction: str,
        reference: Optional[str],
        input: Optional[str],
    ) -> dict:
        # Get the standard input dictionary from the parent class
        eval_input = super()._get_eval_input(prediction, reference, input)

        # 'reference' here is our code snippet, e.g., "len(df)"
        # We execute it to get the live ground truth value.
        # WARNING: Using `eval` can be risky. Only run trusted code.
        live_ground_truth = eval(eval_input["reference"])
        
        # Replace the code snippet with the actual live answer
        eval_input["reference"] = str(live_ground_truth)
        
        return eval_input

In [15]:
# Create an instance of our custom evaluator chain
base_evaluator = DynamicReferenceEvaluator.from_llm(
    criteria="correctness", llm=ChatOpenAI(model="gpt-4", temperature=0)
)

# Wrap it in a LangChainStringEvaluator to map the run/example fields correctly
dynamic_evaluator = LangChainStringEvaluator(
    base_evaluator,
    # This function maps the dataset fields to what our evaluator expects
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["reference_code"],
        "input": example.inputs["question"],
    },
).as_run_evaluator()

# Run the evaluation at Time "T1"
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=predict_pandas_agent,
    evaluation=RunEvalConfig(
        custom_evaluators=[dynamic_evaluator],
    ),
    project_metadata={"time": "T1"},
    max_concurrency=1, # Pandas agent isn't thread-safe
)

/var/folders/40/2qwvhqqd5x38g72sq_ktmlzm0000gn/T/ipykernel_27519/1330203278.py:18: LangChainDeprecationWarning: The following arguments are deprecated and will be removed in a future release: dict_keys(['max_concurrency']).
  client.run_on_dataset(


View the evaluation results for project 'dear-click-47' at:
https://smith.langchain.com/o/447a3e6d-e186-494e-9c8e-c4650326fd0d/datasets/c983fc8c-d1a2-4d58-98a5-9ab775f64b0c/compare?selectedSessions=d7f19481-0e8d-4e2e-892b-c026606606a8

View all tests for Dataset Dynamic Titanic QA at:
https://smith.langchain.com/o/447a3e6d-e186-494e-9c8e-c4650326fd0d/datasets/c983fc8c-d1a2-4d58-98a5-9ab775f64b0c


Chain failed for example 81327d10-4780-4ebf-a87c-0a8177f7292c with inputs {'question': 'How many passengers survived?'}
Error Type: NameError, Message: name 'create_pandas_dataframe_agent' is not defined
Chain failed for example 51abab2f-a113-4e66-b768-0b7c5c5c53da with inputs {'question': 'What was the average age of the passengers?'}
Error Type: NameError, Message: name 'create_pandas_dataframe_agent' is not defined
Chain failed for example 8bf9e5c9-bf6c-4b91-bf0c-8498073d91c9 with inputs {'question': 'How many passengers were on the Titanic?'}
Error Type: NameError, Message: name 'create_pandas_dataframe_agent' is not defined
Chain failed for example a36f07a4-9b8c-43ff-9fc7-1caae8378fb9 with inputs {'question': 'How many male and female passengers were there?'}
Error Type: NameError, Message: name 'create_pandas_dataframe_agent' is not defined
Chain failed for example ae57634f-a538-4fdd-9535-46492887a870 with inputs {'question': 'What was the average fare paid for the tickets?'}
Err

[------------------------------------------------->] 5/5


{'project_name': 'dear-click-47',
 'results': {'51abab2f-a113-4e66-b768-0b7c5c5c53da': {'input': {'question': 'What was the average age of the passengers?'},
   'feedback': [],
   'execution_time': 0.014431,
   'run_id': '0bf062dc-b6f3-4450-a422-48e3d5093f2b',
   'Error': NameError("name 'create_pandas_dataframe_agent' is not defined"),
   'reference': {'reference_code': "df['Age'].mean()"}},
  '81327d10-4780-4ebf-a87c-0a8177f7292c': {'input': {'question': 'How many passengers survived?'},
   'feedback': [],
   'execution_time': 0.014074,
   'run_id': 'e1a4db5e-ac09-4058-b67a-f93ba6ca6bfc',
   'Error': NameError("name 'create_pandas_dataframe_agent' is not defined"),
   'reference': {'reference_code': "df['Survived'].sum()"}},
  '8bf9e5c9-bf6c-4b91-bf0c-8498073d91c9': {'input': {'question': 'How many passengers were on the Titanic?'},
   'feedback': [],
   'execution_time': 0.012868,
   'run_id': '50d3d0ae-bce4-4caa-b502-59c7105c2990',
   'Error': NameError("name 'create_pandas_datafra